<a href="https://colab.research.google.com/github/megane-chan/RED_dialogue/blob/main/dialogue_v1_milan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
![ -e RED_dialogue ] && rm -r RED_dialogue
!git clone https://github.com/megane-chan/RED_dialogue
!cd RED_dialogue

In [14]:
!pip install -q transformers datasets sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.2 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, Dataset
import pandas as pd

ds = load_dataset("daily_dialog", "train")


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [5]:
label_names =  ["dummy", "inform", "question", "directive", "commissive"]#, "no emotion", "anger", "disgust", "fear", "happiness"]
id2label = {idx:label for idx, label in enumerate(label_names)}
label2id = {label:idx for idx, label in enumerate(label_names)}


In [6]:
def dataprep(train):
  #train['dialog'] =  [item for sublist in train['dialog'] for item in sublist]

  encoding = tokenizer.batch_encode_plus(train['dialog'], padding="max_length", truncation=True, max_length=128)
  all_labels = []


  for j in range(0, len(encoding["input_ids"])-1):
    labels = np.zeros((128,len(label_names)))
    labels[:, train['act'][j]] = 1
    
    #labels[:, train['emotion'][j] + 5 ] = 1
    all_labels = all_labels + [labels.tolist()]

  encoding['labels'] = all_labels
  return encoding

Idea: introduce a learned dialog embedding which is learned, but it allows you to match the similarity of different dialog interations. Calculated similar to sentance enmbedding, using a self-attention head to learn dialog embeddings  using \[MASK] token to hide sentances and guessing which sentances are hidden. Essentially, what BERT does to sentances on a dialog level


I googled a little and it already exists!!!! Yay.


https://arxiv.org/abs/2012.01775 - DialogBERT: uses the dialog embedding that I was thinking about. Also uses a very similar learning methodology to what I was thinking about.

https://aclanthology.org/N19-1373/ - Similar but uses bidirectional RNN's instead of bert encoder model. Does use self-attention

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
})

In [7]:
encoded = ds.map(dataprep)

Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
#encoded['train'][0]
ds['train'][0]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ',
  ' You know that is tempting but is really not good for our fitness . ',
  ' What do you mean ? It will help us to relax . ',
  " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
  " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
  ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
  " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
  ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
  " Good.Let ' s go now . ",
  ' All right . '],
 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4],
 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4]}

In [ ]:
encoded.set_format("torch")

In [ ]:
encoded

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [8]:
batch_size = 8
metric_name = "f1"
#from transformers import LongformerForSequenceClassification
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="single_label_classification", 
                                                           num_labels=len(label_names),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)


In [9]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)

    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
encoded['train'][0]['labels'].type()


'torch.FloatTensor'

In [ ]:
encoded['train']['input_ids'][0]

tensor([[ 101, 2360, 1010,  ...,    0,    0,    0],
        [ 101, 2017, 2113,  ...,    0,    0,    0],
        [ 101, 2054, 2079,  ...,    0,    0,    0],
        ...,
        [ 101, 4165, 2307,  ...,    0,    0,    0],
        [ 101, 2204, 1012,  ...,    0,    0,    0],
        [ 101, 2035, 2157,  ...,    0,    0,    0]])

In [ ]:
#forward pass
#outputs = model(input_ids=encoded['train']['input_ids'][0][0].unsqueeze(0), labels=encoded['train'][0]['labels'].unsqueeze(0))
#outputs

#encoded['train'][0]['labels'].unsqueeze(0)
#encoded['train']['input_ids'][0].unsqueeze(0)

tensor([[[0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.]]])

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')



In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = [item for sublist in ds['train']['dialog'] for item in sublist]
sentences = sentences[1:10] + sentences[99:102]
# Load model from HuggingFace Hub

# change to a diff pretainered model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=len(sentences), dim=1)

sentence_embeddings

# cosine similarity of sentance embeddings is a measure of conflict or coherance


tensor([[-0.1171,  0.1119, -0.1163,  ..., -0.8604, -0.0085, -0.0262],
        [ 0.0773, -0.1501,  0.1615,  ...,  0.0172, -0.2544,  0.0420],
        [-0.1471, -0.3525,  0.3902,  ..., -0.2781,  0.0501,  0.1924],
        ...,
        [-0.3628, -0.0131, -0.0228,  ..., -0.0093,  0.4305,  0.4399],
        [ 0.3949,  0.2582,  0.4420,  ..., -0.2152, -0.6678, -0.0894],
        [-0.6683, -0.0887,  0.3317,  ...,  0.2974, -0.4025, -0.2931]])

In [24]:
import numpy as np
print(sentences[1])
print(sentences[2])


#cosine similary might not work. This has a high similarty, but is conflict
sim1 = np.dot(sentence_embeddings[1],sentence_embeddings[2])/(np.linalg.norm(sentence_embeddings[1]) * np.linalg.norm(sentence_embeddings[2]))
print(sim1)

#no conflict here, but lower score than the high conflict one
print(sentences[6])
sim2 = np.dot(sentence_embeddings[1],sentence_embeddings[6])/(np.linalg.norm(sentence_embeddings[1]) * np.linalg.norm(sentence_embeddings[6]))
sim2

 What do you mean ? It will help us to relax . 
 Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? 
0.35189167
 Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . 


0.23882763